## Simple RAG Pipeline Evaluation
In this notebook we will create a simple rag pipeline with basic strategies to determine the baseline result.
These are the strategies we are following in this simple rag pipeline
- Recursive Character Splitter with 1000 chunk size and chunk overlap of 250
- OpenAI default embedding model: text-embedding-ada-002
- Vector store as the retriever 
- Generator Model: gpt4o

We are using RAGS for evaluation. The choosen RAGAS metrics are:
- Faithfulness
- Context Recall
- Answer Relevancy
- Context Precision

In [2]:
# Importing libraries
import sys
from dotenv import load_dotenv
import json
import pandas as pd
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
sys.path.insert(1, '/home/jabez/week_11/Contract-Advisor-RAG')
load_dotenv()
sys.path.insert(1, '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/scripts')
import file_loader 
import pipelines 
import evaluation

In [3]:
# Load JSON from file
json_path = '../filepath.json'

with open(json_path, 'r') as json_file:
    file_paths = json.load(json_file)
data_file_path = file_paths['data_file_path']
synthetic_test_data_path = file_paths['synthetic_test_data_path']

# loading data
data = file_loader.load_csv(data_file_path)

# loading synthetic test data
synthetic_test_data = pd.read_csv(synthetic_test_data_path)

# loading persist directory for deafult vector db
persist_directory_for_default_vector_db = file_paths['persist_directory_for_default_vector_db']

In [4]:
# Load a Chroma database
embeddings = OpenAIEmbeddings()
smaller_chunck_vector_db = Chroma(persist_directory=persist_directory_for_default_vector_db, embedding_function=embeddings)

In [5]:
# Setting retriever
retriver = smaller_chunck_vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [6]:
# Adding answer to test data from simple pipeline
syntetic_test_data_with_answer = evaluation.adding_answer_to_testdata(synthetic_test_data, pipelines.simple_pipeline, smaller_chunck_vector_db, retriver)

/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Failed to batch ingest runs: LangSmithRateLimitError('Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError(\'429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Monthly unique traces usage limit exceeded"}\')')
Failed to batch ingest runs: LangSmithConnectionError("Connection error caused failure to POST https://api.smith.langchain.com/runs/batch  in LangSmith API. Please confirm your internet connection.. ConnectionError(ProtocolError('Connection aborted.', TimeoutError('The write operation timed out')))")
Failed to batch ingest runs: LangSmithConnectionError("Connection error caused failure to 

Failed to batch ingest runs: LangSmithConnectionError('Connection error caused failure to POST https://api.smith.langchain.com/runs/batch  in LangSmith API. Please confirm your internet connection.. SSLError(MaxRetryError("HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Max retries exceeded with url: /runs/batch (Caused by SSLError(SSLEOFError(8, \'EOF occurred in violation of protocol (_ssl.c:2426)\')))"))')
Failed to batch ingest runs: LangSmithConnectionError("Connection error caused failure to POST https://api.smith.langchain.com/runs/batch  in LangSmith API. Please confirm your internet connection.. ConnectionError(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))")
Failed to batch ingest runs: LangSmithConnectionError('Connection error caused failure to POST https://api.smith.langchain.com/runs/batch  in LangSmith API. Please confirm your internet connection.. SSLError(MaxRetryError("HTTPSConnectionPool(hos

In [7]:
# Evaluating the test data from simple pipeline
simple_rag_evaluation_result = evaluation.ragas_evaluator(syntetic_test_data_with_answer)

Evaluating: 100%|██████████| 80/80 [00:57<00:00,  1.40it/s]


In [9]:
# Evaluation mean
result = evaluation.evaluation_mean(simple_rag_evaluation_result)

context_precision: 86.79%, faithfulness: 79.29%, answer_relevancy: 91.73%, context_recall: 73.75%


In [10]:
simple_rag_evaluation_result

,question,answer,contexts,ground_truth,context_precision,faithfulness,answer_relevancy,context_recall
0,What role did Actor Benedict Cumberbatch play ...,"Actor Benedict Cumberbatch, who is a distant c...","[for the long-dead King, swinging incense and ...",Actor Benedict Cumberbatch read a poem at the ...,0.966667,0.750000,0.882254,1.000000
1,What was the connection between Ibragim Todash...,Ibragim Todashev was an associate of Tamerlan ...,[highlights: Ibragim Todashev was killed durin...,Ibragim Todashev was an associate of Boston Ma...,1.000000,1.000000,0.913261,1.000000
2,"What are some pros and cons of the movie ""Pret...","The movie ""Pretty Woman"" has several pros and ...",[article: (CNN)For a generation of moviegoers ...,"The movie ""Pretty Woman"" has some pros such as...",1.000000,1.000000,0.999299,1.000000
3,How does Parisa Tabriz contribute to cybersecu...,Parisa Tabriz contributes to cybersecurity at ...,[highlights: Parisa Tabriz is the 31-year-old ...,Parisa Tabriz contributes to cybersecurity at ...,1.000000,0.666667,0.990183,0.750000
4,What is Maurizio Cattelan known for in the art...,Maurizio Cattelan is known for his humorous an...,"[Known for his humorous and satirical art, Cat...",Maurizio Cattelan is known for his humorous an...,0.833333,0.666667,0.991169,1.000000
5,How have foreign tourists been affected by Isl...,Foreign tourists in parts of North and sub-Sah...,[article: (CNN)The attack on Tunisia's famed B...,Foreign tourists have been affected by Islamis...,1.000000,1.000000,0.989343,0.800000
6,What are potential buyers asking about regardi...,"Potential buyers are asking ""lots of serious q...","[only claim to fame, Sells says. The former ma...","Potential buyers are asking ""lots of serious q...",0.916667,1.000000,0.955961,1.000000
7,What are the responsibilities and goals of Aym...,"As the leader of the opposition in Israel, Aym...",[highlights: Ayman Odeh is a new face in polit...,Ayman Odeh's responsibilities as the leader of...,1.000000,0.833333,0.994708,0.833333
8,How could the memristor revolutionize electron...,The memristor could revolutionize electronics ...,"[At the moment, manufacturing costs are still ...",The memristor could revolutionize electronics ...,1.000000,1.000000,0.961102,1.000000
9,What are some features of the city of Granada ...,Some features of the city of Granada that cont...,[boutiques are full of local art and street ca...,"Colonial buildings, narrow alleyways, garden c...",0.950000,0.500000,0.998726,1.000000


In [ ]:
simple_rag_evaluation_result.to_csv('/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/test_data/simple_rag_evaluation_result.csv', index=False)